### 🚀 Objective-Driven Web Crawling with WaterCrawl, LiteLLM, and Rank-BM25

Welcome to this **step-by-step Jupyter Notebook** where we explore goal-oriented web crawling. This tutorial will guide you through using WaterCrawl to search and scrape web content, filter URLs with Rank-BM25 and LLMs, and analyze data to meet a specific objective for a company or website.

#### What’s inside?
| 🔧 Component | 💡 Why we’re using it |
|--------------|----------------------|
| **WaterCrawl** | For searching the web and scraping content with precision. |
| **LiteLLM** | To interact with various LLM providers for strategy generation, ranking, and content analysis. |
| **Rank-BM25** | For efficient keyword-based URL filtering and ranking. |

#### Notebook Flow 🗺️
1. **Setup**: Install dependencies and configure API keys.
2. **Initialization**: Set up the target company or URL and objective.
3. **Search Strategy Generation**: Use an LLM to create search strategies.
4. **Web Search**: Perform a search with WaterCrawl to gather URLs.
5. **URL Filtering**: Apply Rank-BM25 and LLM-based filtering to select relevant URLs.
6. **Content Scraping**: Scrape content from top URLs using WaterCrawl.
7. **Content Analysis**: Analyze scraped content with LLMs to meet the objective.
8. **Results**: Compile and display the final structured response.

#### Why you’ll ❤️ this approach
- **Efficiency**: Quickly process vast web data.
- **Flexibility**: Switch between LLM providers for different tasks.
- **Precision**: Combine keyword and semantic analysis for accurate results.
- **Customizability**: Tailor search depth, language, and model selection.

> **Tip:** If you’re new to WaterCrawl, check out the [WaterCrawl Documentation](https://docs.watercrawl.dev/intro) for more details.

Ready? Let’s start by setting up our environment! 🏁

##### ➡️ **Install all the dependencies:**

In [4]:
# !pip install -r requirements.txt

##### ➡️ **API keys you’ll need (grab these first!)** 

| Service | What it’s for | Where to generate |
|---------|---------------|-------------------|
| **WaterCrawl** | Auth for search and crawling endpoints | <https://app.watercrawl.dev/dashboard/api-keys> |
| **OpenAI/Other LLMs** | LLM interactions | Depends on provider (e.g., OpenAI, Anthropic) |

---
**Option 1 – Keep it clean: Use a `.env` file** ⚠️

Create the file **once**, store your keys, and everything else “just works”.

```python
# Create .env file
env_text = """
WATERCRAWL_API_KEY=your_watercrawl_api_key_here
OPENAI_API_KEY=your_openai_api_key_here
ANTHROPIC_API_KEY=your_anthropic_api_key_here
DEEPSEEK_API_KEY=your_deepseek_api_key_here
""".strip()

with open(".env", "w") as f:
    f.write(env_text)
print(".env file created — now edit it with your real keys ✏️")
```

**Option 2 – Quick-and-dirty: Hard-code in the notebook** ⚠️

Not recommended — anyone who sees or commits the notebook can read your keys.

WATERCRAWL_API_KEY=your_watercrawl_api_key_here
LITELLM_API_KEY=your_litellm_api_key_here

##### ➡️ **If you’re using a `.env` file, load the API keys with dotenv**

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()  # pulls everything from .env for any API keys for LLMS

WATERCRAWL_API_KEY = os.environ.get("WATERCRAWL_API_KEY")


##### ➡️ **Import necessary packages:**

In [2]:
import sys
import os
sys.path.append(os.path.abspath('./objective_crawler'))
from objective_crawler.core import ObjectiveCrawler
from objective_crawler.clients import WaterCrawler, LLMClient
from objective_crawler.config import DEFAULT_SEARCH_LLM, DEFAULT_RANKING_LLM, DEFAULT_CONTENT_ANALYSIS_LLM, DEFAULT_REASONING_LLM, DEFAULT_TOP_K, DEFAULT_NUM_SEARCH_STRATEGIES, DEFAULT_RELEVANCE_THRESHOLD, DEFAULT_SEARCH_DEPTH

#### ➡️ **Set up the Input and Objective:**
Define the company name or website URL you want to search for and the objective you want to achieve.

In [3]:
# Example Input and Objective
company_or_url = "Tesla"
objective = "Find information about stock value"
use_map = False
# You can modify these to test with your own inputs
print(f"Input: {company_or_url}")
print(f"Objective: {objective}")


Input: Tesla
Objective: Find information about stock value


#### ➡️ **Initialize the Crawler:**
Set up the crawler with configurable options for LLM models, number of URLs to scrape, search strategies, and search depth.

### Models

<div style="display: flex; justify-content: space-between; flex-wrap: wrap; gap: 10px;">

<div style="flex: 1; min-width: 200px; background-color: #f0f0f0; padding: 10px; border-radius: 5px;">

**Anthropic Models:**
- anthropic/claude-sonnet-4-20250514
- anthropic/claude-opus-4-20250514
- anthropic/claude-3-7-sonnet-20250219
- anthropic/claude-3-5-sonnet-20240620
- anthropic/claude-3-sonnet-20240229
- anthropic/claude-3-haiku-20240307
- anthropic/claude-3-opus-20240229

</div>

<div style="flex: 1; min-width: 200px; background-color: #f0f0f0; padding: 10px; border-radius: 5px;">

**DeepSeek Models:**
- deepseek/deepseek-chat
- deepseek/deepseek-coder

</div>

<div style="flex: 1; min-width: 200px; background-color: #f0f0f0; padding: 10px; border-radius: 5px;">

**OpenAI Models:**
- o4-mini
- o3
- o3-mini
- o1-mini
- o1-preview
- gpt-4.1
- gpt-4.1-mini
- gpt-4.1-nano
- gpt-4o
- gpt-4o-mini

</div>

</div>


## Customizing Your Web Crawling Experience: Key Configuration Options

When using an objective-driven web crawling tool, you can fine-tune various settings to optimize how it searches, evaluates, and interprets information. Each setting comes with **default values** defined in a configuration file ([config.py](cci:7://file:///c:/Users/armof/projcects/langchain/watercrawl-tutorials/URL%20and%20objective%20%28Map,%20filter,%20and%20scrape%29/objective_crawler/config.py:0:0-0:0)), which are used automatically if you don’t specify custom values. As a user, you have the flexibility to stick with these defaults for simplicity or override them with new values to tailor the tool to your specific needs. Below, I’ll break down each configuration option in simple terms, highlighting the defaults and the custom settings shown.

- **SEARCH_LLM (Default: "o4-mini", Custom: "deepseek/deepseek-chat")**  
  This setting chooses the language model that generates search queries to find relevant information. Imagine it as the creative mind brainstorming the best search terms for your goal. The default is set to "o4-mini" in the config, but here it’s customized to DeepSeek's chat model, which excels at crafting diverse and targeted search ideas to kickstart the process.

- **RANKING_LLM (Default: "o4-mini", Custom: "anthropic/claude-sonnet-4-20250514")**  
  After gathering potential web pages, this model ranks them based on how likely they are to contain useful content. Think of it as a discerning judge prioritizing the most relevant search results. The default is "o4-mini," but it’s overridden here to Anthropic's Claude Sonnet (version 4-20250514), a powerful choice for accurate and nuanced ranking.

- **CONTENT_ANALYSIS_LLM (Default: "o4-mini", Custom: "gpt-4.1-mini")**  
  Once web pages are scraped, this model dives into the content, extracting key details tied to your objective. Picture it as a meticulous researcher summarizing critical insights from each page. By default, it uses "o4-mini," but this example sets it to OpenAI's GPT-4.1 Mini, offering a balance of efficiency and depth for analyzing text.

- **REASONING_LLM (Default: "o4-mini", Custom: "o3")**  
  This is the final brain that compiles all the analyzed data into a coherent answer or summary for your objective. It’s like the writer of your final report, weaving together insights from multiple sources. The config defaults to "o4-mini," but it’s changed here to "o3," likely referring to an OpenAI model (possibly a variant or placeholder), chosen for its strong synthesis and reasoning capabilities.

- **TOP_K (Default: 5, Custom: "4")**  
  This number decides how many top-ranked web pages will be deeply analyzed. It’s akin to selecting the top articles to study closely from a broader list. The default in the config is 5, but setting it to 4, as shown, ensures a focused analysis on a slightly smaller, high-quality set of sources, balancing depth with efficiency.

- **STRATEGIES (Default: 3, Custom: "3")**  
  This determines how many different search approaches or query variations the tool will use to find relevant pages. Think of it as asking different experts to search in their unique styles. The default is 3 in the config, and this example keeps it at 3, striking a balance by providing variety in search methods without overwhelming the process.

- **SEARCH_DEPTH (Default: "basic", Custom: "basic")**  
  This controls the intensity of the web search—how deep the tool digs into the internet. It’s like choosing between a quick skim or an exhaustive dive. The default is "basic" in the config, and it’s left as "basic" here, opting for a faster, less intensive search, ideal for straightforward goals, saving time and resources while still delivering results.

By tweaking these settings, you can tailor the web crawling tool to match your specific needs—whether you’re aiming for speed, precision, or comprehensive coverage. You can rely on the defaults from the configuration file for a ready-to-go setup or set custom values, as shown in this example, to experiment and optimize the journey from raw web data to actionable insights!

In [4]:

#We can use the default values:
# SEARCH_LLM= DEFAULT_SEARCH_LLM
# RANKING_LLM= DEFAULT_RANKING_LLM
# CONTENT_ANALYSIS_LLM= DEFAULT_CONTENT_ANALYSIS_LLM
# REASONING_LLM= DEFAULT_REASONING_LLM
# TOP_K=DEFAULT_TOP_K
# STRATEGIES=DEFAULT_NUM_SEARCH_STRATEGIES
# SEARCH_DEPTH=DEFAULT_SEARCH_DEPTH
# ...

# Or Selecting the  values 
SEARCH_LLM= "deepseek/deepseek-chat"
RANKING_LLM= "anthropic/claude-sonnet-4-20250514"
CONTENT_ANALYSIS_LLM= "gpt-4.1-mini"
REASONING_LLM= "o3"
TOP_K= 5    
NUM_SEARCH_STRATEGIES=3
SEARCH_DEPTH="basic"
MAX_CONTENT_CHARS = 6000  # amount of scraped markdown passed back to GPT
DEBUG_MODE="False"
# Search-related constants
SEARCH_RESULT_LIMIT = 20  # 4 * DEFAULT_TOP_K (get more results initially for filtering)
BM25_RESULT_COUNT = 10  # 2 * DEFAULT_TOP_K (intermediate filtering step)
SEARCH_DEPTH = "basic"  # Options: "basic", "advanced", "ultimate"
SEARCH_LANGUAGE = None  # None means auto-detect
SEARCH_COUNTRY = None  # None means auto-detect

In [5]:

# Initialize WaterCrawler with your API key
water_crawler = WaterCrawler(api_key=WATERCRAWL_API_KEY)

# Create ObjectiveCrawler instance
crawler = ObjectiveCrawler(
    wc = water_crawler,
    top_k =TOP_K,
    num_search_strategies = NUM_SEARCH_STRATEGIES,
    search_result_limit = SEARCH_RESULT_LIMIT,
    bm25_result_count = BM25_RESULT_COUNT,
    search_depth = SEARCH_DEPTH,
    search_language = SEARCH_LANGUAGE,
    search_country = SEARCH_COUNTRY,
    debug_mode = DEBUG_MODE,
    search_model = SEARCH_LLM,
    ranking_model = RANKING_LLM,
    reasoning_model = REASONING_LLM,
    content_analysis_model = CONTENT_ANALYSIS_LLM  
)

print("Crawler initialized with models:")
print(f"Search LLM: {SEARCH_LLM}")
print(f"Ranking LLM: {RANKING_LLM}")
print(f"Content Analysis LLM: {CONTENT_ANALYSIS_LLM}")
print(f"Reasoning LLM: {REASONING_LLM}")
print(f"Top K URLs to scrape: {TOP_K}")
print(f"Number of search strategies: {NUM_SEARCH_STRATEGIES}")
print(f"Search Depth: {SEARCH_DEPTH}")

[INFO] Initialized WaterCrawler


Crawler initialized with models:
Search LLM: deepseek/deepseek-chat
Ranking LLM: anthropic/claude-sonnet-4-20250514
Content Analysis LLM: gpt-4.1-mini
Reasoning LLM: o3
Top K URLs to scrape: 5
Number of search strategies: 3
Search Depth: basic


#### ➡️ **Generate Search Strategies with LLM:**
Use a dedicated LLM to generate search strategies for finding relevant URLs based on the objective.

In [7]:
# Generate search strategies
strategies = crawler._derive_search_strategies(objective, company_or_url)

print("Generated Search Strategies:")
for i, strategy in enumerate(strategies, 1):
    print(f"{i}. {strategy}")

22:53:33 - LiteLLM:INFO: utils.py:3225 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
[INFO] 
LiteLLM completion() model= deepseek-chat; provider = deepseek
[INFO] HTTP Request: POST https://api.deepseek.com/beta/chat/completions "HTTP/1.1 200 OK"
22:53:41 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
[INFO] Wrapper: Completed Call, calling success_handler
[INFO] Search strategies: ['Tesla stock value', 'site:tesla.com stock price', 'Tesla AND (stock OR share) value']


DEBUG: SEARCH STRATEGIES GENERATED
[DEBUG] 1. 'Tesla stock value'
[DEBUG] 2. 'site:tesla.com stock price'
[DEBUG] 3. 'Tesla AND (stock OR share) value'
Generated Search Strategies:
1. Tesla stock value
2. site:tesla.com stock price
3. Tesla AND (stock OR share) value


#### ➡️ **Perform Web Search with WaterCrawl and Filter URLs with BM25 :**
Use WaterCrawl to search the web based on the generated strategies and filter candidate URLs using BM25.

In [9]:
# Perform web search
candidate_urls, all_results = crawler._collect_candidate_urls(company_or_url, use_map, strategies)

print(f"Total URLs from search: {len(candidate_urls)}")
print("Sample URLs:", candidate_urls[:5] if candidate_urls else [])

[INFO] Search 1/3: 'Tesla stock value'


[DEBUG] Sending search query 1/3: 'Tesla stock value'


[INFO] Search request created with ID: 3af8db0a-25cd-46e6-8cbe-e23b464aa6c0
[INFO] Search 'Tesla stock value' returned 20 new results (20 total unique)
[INFO] Search 2/3: 'site:tesla.com stock price'


[DEBUG] Received 20 results, 20 new, total unique URLs: 20
[DEBUG] Sending search query 2/3: 'site:tesla.com stock price'


[INFO] Search request created with ID: 8b76b001-7e61-4ff3-9188-3c98734815eb
[INFO] Search 'site:tesla.com stock price' returned 19 new results (39 total unique)
[INFO] Search 3/3: 'Tesla AND (stock OR share) value'


[DEBUG] Received 20 results, 19 new, total unique URLs: 39
[DEBUG] Sending search query 3/3: 'Tesla AND (stock OR share) value'


[INFO] Search request created with ID: b7458f42-37a9-4995-9e33-fc56e98fefa3
[INFO] Search 'Tesla AND (stock OR share) value' returned 13 new results (52 total unique)


[DEBUG] Received 20 results, 13 new, total unique URLs: 52
DEBUG: BM25 URL RANKING
[DEBUG] Found 10 relevant URLs out of 52.
[DEBUG] Top 10 matches:
[DEBUG]  1. Score: 5.87 | https://ir.tesla.com/node/16
[DEBUG]  2. Score: 4.85 | https://tradingeconomics.com/tsla:us
[DEBUG]  3. Score: 3.75 | https://www.reddit.com/r/OutOfTheLoop/comments/1iyp7dz/whats_up_with_elon_musk_seemingly_not_caring/
[DEBUG]  4. Score: 3.43 | https://www.ark-invest.com/articles/valuation-models/arks-tesla-model
[DEBUG]  5. Score: 3.39 | https://www.npr.org/2023/01/06/1146941980/tesla-shares-elon-musk-twitter-electric-cars
[DEBUG]  6. Score: 3.39 | https://www.ark-invest.com/articles/valuation-models/arks-tesla-price-target-2029
[DEBUG]  7. Score: 3.14 | https://www.nbcnews.com/business/business-news/tesla-falling-stock-resale-value-elon-musk-trump-what-to-know-rcna196497
[DEBUG]  8. Score: 3.13 | https://www.calpers.ca.gov/newsroom/calpers-news/2024/calpers-to-vote-against-tesla-ceo-multibillion-dollar-pay-packa

#### ➡️ **Further Filter URLs with LLM:**
Use an LLM to refine the list of URLs based on semantic relevance to the objective.

In [11]:
# Filter URLs with LLM
final_url_list = crawler._rank_links(candidate_urls, objective, company_or_url)

print(f"URLs after LLM filtering: {len(final_url_list)}")
print("Top URLs for scraping:", final_url_list)

23:06:46 - LiteLLM:INFO: utils.py:3225 - 
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
[INFO] 
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
[INFO] HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
23:06:53 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
[INFO] Wrapper: Completed Call, calling success_handler
[INFO] Top 5 candidate pages:
[INFO] https://ir.tesla.com/node/16                                  1.00  Official Tesla investor relations page - primary source for authoritative stock and financial information directly from the company
[INFO] https://www.cnbc.com/quotes/TSLA                              0.95  CNBC quotes page specifically for TSLA stock ticker - provides real-time stock price, charts, and financial data
[INFO] https://tradingeconomics.com/tsla:us                          0.90  Financial data platform with dedicated TSLA stock page showing current v

DEBUG: URL RANKING RESULTS
[DEBUG] 1. https://ir.tesla.com/node/16
[DEBUG]    Score: 1.00 | Reason: Official Tesla investor relations page - primary source for authoritative stock and financial information directly from the company
[DEBUG] 2. https://www.cnbc.com/quotes/TSLA
[DEBUG]    Score: 0.95 | Reason: CNBC quotes page specifically for TSLA stock ticker - provides real-time stock price, charts, and financial data
[DEBUG] 3. https://tradingeconomics.com/tsla:us
[DEBUG]    Score: 0.90 | Reason: Financial data platform with dedicated TSLA stock page showing current value, historical data, and market analysis
[DEBUG] 4. https://www.ark-invest.com/articles/valuation-models/arks-tesla-price-target-2029
[DEBUG]    Score: 0.85 | Reason: Professional investment firm's Tesla valuation model with specific price targets - highly relevant for stock value analysis
[DEBUG] 5. https://www.ark-invest.com/articles/valuation-models/arks-tesla-model
[DEBUG]    Score: 0.80 | Reason: ARK Invest's compr

#### ➡️ **Scrape Content of Top URLs and Analyze its content, finaly get final answer leveraging Reasoning_llm  :**
Scrape the content of the selected URLs using WaterCraw, then call _analyze_page_content to have the content_llm analyze the content, then get the final answer leveraging reasoning_llm calling _get_final_answer

In [18]:
analysis_results = []
search_results_dict = {r.get("url", ""): r for r in all_results if "url" in r} if not use_map else {}

for i, page_url in enumerate(final_url_list, 1):
    try:
        markdown_content = crawler.scrape(page_url)
        if markdown_content is None:
            raise ValueError(f"Scrape returned None for URL: {page_url}")
        analysis_result = crawler._analyze_page_content(markdown_content, objective, page_url)
    except Exception as e:
        description = search_results_dict.get(page_url, {}).get("description", "No description available")
        analysis_result = {
            "verified_url": page_url,
            "objective": objective,
            "result_of_analysis": f"URL could not be crawled: {page_url}. Description: {description}"
        }
    analysis_results.append(analysis_result)

# Store individual analyses in metadata for debugging or transparency
result = crawler._generate_final_result(analysis_results, objective)
result["_metadata"] = {"individual_analyses": analysis_results}

23:16:54 - LiteLLM:INFO: utils.py:3225 - 
LiteLLM completion() model= o3; provider = openai
[INFO] 
LiteLLM completion() model= o3; provider = openai


DEBUG: FINAL RESULT GENERATION
[DEBUG] Successful analyses: 5
[DEBUG] Failed analyses: 0


[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
23:17:19 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
[INFO] Wrapper: Completed Call, calling success_handler


##### **Printing the Results**:

In [28]:
import json
pretty_json = json.dumps(result, indent=2)
print(pretty_json)


{
  "objective_fulfilled": true,
  "final_answer": "Available public sources provide both real-time market data for Tesla\u2019s stock (ticker: TSLA) and independent valuation targets:\n\n1. Current / Real-Time Market Value\n   \u2022 CNBC\u2019s quote page and TradingEconomics both stream the live price of TSLA together with historical charts, fundamental ratios and news feeds.  These pages are intended for investors who need the up-to-the-second trading value.\n\n2. Independent Valuation Models\n   \u2022 ARK Invest 2029 Target \u2013 ARK estimates a base-case price of roughly $2,600 per share for Tesla in 2029, with a bear case near $2,000 and a bull case near $3,100.\n   \u2022 ARK Invest 2026 Target \u2013 In a separate model, ARK\u2019s expected value per share for 2026 is approximately $4,600, based on its open-source assumptions about vehicle sales, autonomous ride-hail revenue, and margins.\n\n3. Company Investor-Relations Context\n   \u2022 Tesla\u2019s own Investor-Relations

In [36]:
class Colors:
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    ENDC = '\033[0m' # Resets the color
    BOLD = '\033[1m'
    
print(Colors.BOLD + Colors.OKCYAN + 'THE REASONING LLM ANSWER (FINAL ANSWER OF THE WHOLE ANALYSIS):' + Colors.ENDC)
print()
print(Colors.OKGREEN + result['final_answer'] + Colors.ENDC)
print()

THE REASONING LLM ANSWER (FINAL ANSWER OF THE WHOLE ANALYSIS):

Available public sources provide both real-time market data for Tesla’s stock (ticker: TSLA) and independent valuation targets:

1. Current / Real-Time Market Value
   • CNBC’s quote page and TradingEconomics both stream the live price of TSLA together with historical charts, fundamental ratios and news feeds.  These pages are intended for investors who need the up-to-the-second trading value.

2. Independent Valuation Models
   • ARK Invest 2029 Target – ARK estimates a base-case price of roughly $2,600 per share for Tesla in 2029, with a bear case near $2,000 and a bull case near $3,100.
   • ARK Invest 2026 Target – In a separate model, ARK’s expected value per share for 2026 is approximately $4,600, based on its open-source assumptions about vehicle sales, autonomous ride-hail revenue, and margins.

3. Company Investor-Relations Context
   • Tesla’s own Investor-Relations FAQ notes that the company does not comment on 

#### ➡️ **We could have done all above in One Step!**
In the above cells, we walked you trhough the step by step process of how the crawler works. We could have done all above in one step, just by calling the run method of the crawler.

In [6]:
result_of_run = crawler.run(company_or_url, objective)



[INFO] Objective: Find information about stock value for Tesla
23:47:43 - LiteLLM:INFO: utils.py:3225 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
[INFO] 
LiteLLM completion() model= deepseek-chat; provider = deepseek


DEBUG: OBJECTIVE CRAWLER STARTED
[DEBUG] Objective: Find information about stock value
[DEBUG] Target: Tesla
[DEBUG] Search Depth: basic
[DEBUG] Search Language: None
[DEBUG] Search Country: None


[INFO] HTTP Request: POST https://api.deepseek.com/beta/chat/completions "HTTP/1.1 200 OK"
23:47:49 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
[INFO] Wrapper: Completed Call, calling success_handler
[INFO] Search strategies: ['Tesla stock value', 'site:tesla.com stock price', 'Tesla AND (stock OR share) AND (value OR price)']
[INFO] Search 1/3: 'Tesla stock value'


DEBUG: SEARCH STRATEGIES GENERATED
[DEBUG] 1. 'Tesla stock value'
[DEBUG] 2. 'site:tesla.com stock price'
[DEBUG] 3. 'Tesla AND (stock OR share) AND (value OR price)'
[DEBUG] Sending search query 1/3: 'Tesla stock value'


[INFO] Search request created with ID: 9c9ad5bc-906b-4bc4-957b-d22b0b8b6917
[INFO] Search 'Tesla stock value' returned 20 new results (20 total unique)
[INFO] Search 2/3: 'site:tesla.com stock price'


...

23:49:30 - LiteLLM:INFO: utils.py:3225 - 
LiteLLM completion() model= gpt-4.1-mini; provider = openai
[INFO] 
LiteLLM completion() model= gpt-4.1-mini; provider = openai


DEBUG: CONTENT ANALYSIS FOR: https://ir.tesla.com/node/16
[DEBUG] Original content length: 11718 chars
[DEBUG] Content sent to LLM: 6000 chars (truncated: True)
===== CONTENT SENT TO LLM FOR ANALYSIS =====
[Content truncated to 2000 chars]
[Skip to main content](main-content)

Stock Show All Hide All

  * How do I purchase shares of Tesla? Do you have a Direct Stock Purchase Plan?

Tesla’s shares trade on the NASDAQ exchange, under the ticker symbol TSLA. To purchase shares, you will need to do so through a broker. If you do not have a brokerage account, you will need to open one. At this time, Tesla does not have a direct stock purchase program.

  * Can Tesla comment on a move in its share price or provide investment advice?

Unfortunately, we cannot comment on moves in our share price or offer investment advice.

  * Does Tesla pay a dividend? Does it plan to?

Tesla has never declared dividends on our common stock. We intend on retaining all future earnings to finance future growth

[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
23:49:35 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
[INFO] Wrapper: Completed Call, calling success_handler
23:49:35 - LiteLLM:INFO: utils.py:3225 - 
LiteLLM completion() model= o3; provider = openai
[INFO] 
LiteLLM completion() model= o3; provider = openai


===== LLM ANALYSIS RESULT =====
Relevant: Tesla’s shares trade on NASDAQ under ticker TSLA; no direct stock purchase plan exists, shares must be bought via a broker. IPO was on June 29, 2010, priced at $17 per share. Tesla has never paid dividends and does not anticipate paying any in the foreseeable future. The CUSIP number for common stock is 88160R101. Quarterly financial report dates and SEC filings, which include stock value information, are posted at ir.tesla.com and sec.gov. Tesla does not comment on share price movements or provide investment advice.
===== END LLM ANALYSIS RESULT =====
DEBUG: FINAL RESULT GENERATION
[DEBUG] Successful analyses: 5
[DEBUG] Failed analyses: 0


[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
23:49:57 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
[INFO] Wrapper: Completed Call, calling success_handler
[INFO] Execution time: 134.39 seconds


In [7]:
class Colors:
    OKCYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    GREEN = '\033[92m'
    OKGREEN = '\033[92m'
    ENDC = '\033[0m' # Resets the color
    BOLD = '\033[1m'
    
print(Colors.BOLD + Colors.DARKCYAN + 'THE REASONING LLM ANSWER (FINAL ANSWER OF THE WHOLE ANALYSIS):' + Colors.ENDC)
print()
print(Colors.GREEN + result_of_run['final_answer'] + Colors.ENDC)
print()

THE REASONING LLM ANSWER (FINAL ANSWER OF THE WHOLE ANALYSIS):

Current Tesla, Inc. (TSLA) stock snapshot (as of market close on Monday, July 7, 2025):

Price & Daily Movement
• Closing price: $293.94
• Daily change: −$21.41 (−6.79%)
• After-hours/overnight: $297.02 (+1.05%)

Trading Range & Volume
• 52-week range: $182.00 – $488.54
• Day’s volume: ~118–130 million shares (vs. 10-day avg. ~110 million; 3-month avg. ~116 million)

Market Capitalization & Shares
• Market cap: ≈$946 billion
• Shares outstanding/float: ~3.22 billion / 2.80 billion

Key Valuation Multiples
• Trailing P/E: 161–173
• Forward P/E (NTM): 147–152
• PEG ratio: 5.14
• Price/Sales: 10.77
• Price/Book: 12.68

Profitability & Financial Metrics
• EPS (TTM): $1.76–1.99
• Net margin: 6.4–6.7 %
• Return on equity: ~9 %
• Revenue (TTM): $95.7–97.7 billion
• Net income (TTM): $6.1–7.1 billion
• EBITDA (TTM): $12.65 billion
• Gross margin: 17.7 %
• Cash on hand: ≈$37 billion
• Debt/Equity: 10–17 %
• Levered free cash flow: 

### 🌟 Conclusion
Congratulations! You've successfully used WaterCrawl, LiteLLM, and Rank-BM25 to search the web, filter URLs, scrape content, and analyze it to meet a specific objective for a company or website. 

#### What you’ve learned:
- How to set up and configure tools for web data extraction.
- Generating search strategies and performing web searches with WaterCrawl.
- Filtering URLs with BM25 and LLMs for relevance.
- Scraping and analyzing content to answer targeted questions.

#### Next Steps:
- Experiment with different companies, URLs, and objectives.
- Try different LLM models for each task (search, ranking, analysis, reasoning).
- Scale up by adjusting search depth or integrating with other tools.

If you found this tutorial helpful, consider starring the [WaterCrawl repo](https://github.com/watercrawl/watercrawl) on GitHub! ⭐